# City Comparison of Vancouver - Paris

### Idea
With many baby boomers entering retirement and the world becoming more accessible through globalization, people have more options on where they decide to live and retire. 
This notebook will analyze where one might want to settle in and retire through a city comparison in multiple dimensions.

I've selected two very popular destinations for city dwellers: Vancouver, Canada and Paris, France.

**Problem**: These cities are arguably two of the best cities in the world. How do we objectively decide which one is the best place to live?

**Goal**: This notebook aims to help people better understand what both cities have to offer in a data-supported analysis.

### Vancouver
- Vancouver has the highest population density in Canada, with over 5,400 people per square kilometre.
- Because of its proximity to both mountains and ocean, Vancouver is famous for offering every kind of outdoor sport and adventure, from skiing and snow sports in the winter to kayaking and water sports in the summer, and hiking, camping, and biking year-round. Also fishing, scenic golf courses, and mountain ziplining.
- Vancouver is consistently named as one of the top five worldwide cities for livability and quality of life.

### Paris
- Paris is the capital and most populous city of France, with an estimated population of 2,175,601 residents as of 2018, in an area of more than 105 square kilometres (41 square miles).
- In 2018, Paris was the second most expensive city in the world, after Singapore.
- Since the 17th century, Paris has been one of Europe's major centres of finance, diplomacy, commerce, fashion, gastronomy, science and arts.
- It was ranked as the second most visited travel destination in the world in 2019, after Bangkok and just ahead of London.

### Similarities
- Both are the most expensive cities to live in within their respective countries.
- Both cities have the highest population density in their respective countries.
- Both have an overall climate throughout the year as mild and moderately wet.


_https://www.tourismvancouver.com/_; 
_https://en.wikipedia.org/wiki/Vancouver_; 
_https://en.wikipedia.org/wiki/Paris_; 
_https://www.lonelyplanet.com/france/paris_

## Data Dimensions

- Neighborhoods & Locations: 

| Postal Code | City | Neighborhood | Neighborhood Latitude | Neighborhood Longitude |
| --- | --- | --- | --- | --- |
| V6B | Vancouver | (NE Downtown / Gastown / Harbour Centre / International Village / Victory Square / Yaletown) | -43.55555 | 75.55555 |
| V6C | Vancouver | Waterfront / Coal Harbour / Canada Place) | -43.55555 | 75.5556 |

- Venues: using Foursquare's location data to identify and view the venue counts and their categories. An ideal location is one that has many options for entertainment, food, and more.

| Postal Code | Venue | Venue Latitude | Venue Longitude | Venue Category |
| --- | --- | --- | --- | --- |
| V6B | Sample Coffee | -44.55555 | 75.55555 | Coffee Shop |
| 75001 | Sample Grocery | -44.55555 | 75.55555 | Grocery |

- Population Density: There are correlations between density and increase of options within a city.
- Walkability score: For each address, Walk Score analyzes hundreds of walking routes to nearby amenities. Points are awarded based on the distance to amenities in each category. Amenities within a 5 minute walk (.25 miles) are given maximum points. Walk Score also measures pedestrian friendliness by analyzing population density and road metrics such as block length and intersection density. It is important for retirees to have accessible amenities.
- Number of transit / subway lines: The more transit lines a city has, the easier it is to move around.
- cost of living: It is important for retirees to understand the cost of living to maintain their selected lifestyle.

| PostalCode | Cost Of Living | Population Density | Walkability Score | Transit |
| --- | --- | --- | --- | --- |
| Vancouver | 2200 | 5500 | 94 | 2 |
| Paris | 2500 | 21000 | 100 | 6 |

*Demonstrative data shown above.*